<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/logic/repair_controller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Navigate to project folder and ensure we're on the 'develop' branch
import os, subprocess, sys

REPO_DIR = "/content/Cloud_Computing"
if not os.path.isdir(REPO_DIR):
    subprocess.run([
        "git", "clone", "-b", "develop",
        "https://github.com/Method-for-Software-System-Development/Cloud_Computing.git",
        REPO_DIR
    ], check=True)

# Update & checkout develop if repo already exists
subprocess.run(["git", "-C", REPO_DIR, "fetch", "origin"], check=True)
subprocess.run(["git", "-C", REPO_DIR, "checkout", "develop"], check=True)
subprocess.run(["git", "-C", REPO_DIR, "pull"], check=True)

# Install importnb (only needed once per session)
%pip install importnb --quiet

# Add firebase folder to Python path and import its notebook as module
sys.path.append(os.path.join(REPO_DIR, "firebase"))
from importnb import Notebook

with Notebook():
    import FireBase as fb

from datetime import datetime



In [5]:
# Calculate score for a fault based on severity and time since it was reported
def calculate_fault_score(fault):
    severity_multipliers = {
        "Low": 1,
        "Medium": 2,
        "High": 3,
        "Critical": 5
    }

    severity = fault.get("severity", "Low")
    multiplier = severity_multipliers.get(severity, 1)

    try:
        fault_time = datetime.fromisoformat(fault["timestamp"])
        hours_passed = max(1, round((datetime.now() - fault_time).total_seconds() / 3600))
    except Exception:
        hours_passed = 1  # Default to 1 hour if timestamp is invalid

    return multiplier * hours_passed


# Handle the full repair process: fetch fault, calculate score, update user and fault
def handle_fault_repair(sensor: str, username: str):
    """
    Repairs a fault and updates the user's score accordingly.

    Steps:
    1. Retrieve fault data
    2. Calculate the fault score based on severity and elapsed time
    3. Mark the fault as repaired
    4. Update the user's total score in Firebase

    Returns a status message.
    """
    # Step 1: Get the fault details
    code, fault = fb.get_fault(sensor)
    if code != 200 or fault is None:
        return f"Fault '{sensor}' not found."

    # Step 2: Calculate the score for repairing this fault
    score = calculate_fault_score(fault)

    # Step 3: Mark the fault as repaired in Firebase
    code, updated_fault = fb.mark_fault_as_repaired(sensor, username)
    if code != 200:
        return f"Failed to mark fault as repaired: {updated_fault.get('error', 'Unknown error')}"

    # Step 4: Retrieve the user data
    code, user_data = fb.get_user(username)
    if code != 200 or user_data is None:
        return f"User '{username}' not found."

    # Step 5: Update the user's score
    current_score = user_data.get("score", 0)
    new_score = current_score + score
    fb.update_user(username, {"score": new_score})

    return f"Fault '{sensor}' repaired by {username}. Awarded {score} points (Total: {new_score})."
